# **자동 차별화**
신경망 훈련시 자주 사용되는 알고리즘은 **역전파**이다.    
역전파에서 매개변수는 주어진 매개변수에 대한 손실 함수의 **기울기**에 따라 조정된다.   
**파이토치는 그라이디언트 계산을 위해 `torch.autograd`를 지원한다.**

In [3]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)  # 네트워크의 가중치와 편향 최적화를 위해 `requires_grade_(True)` 속성을 설정한다.
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

### **그라디언트 계산**
신경망에서 매개변수의 가중치를 최적화 하려면 매개변수와 관련해 손실함수의 도함수를 계산해야한다.   
이러한 미분을 계산하기 위해 `loss.backward()` 호출 후 `w.grad` 및 `b.grad`로 값을 검색한다

In [4]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.0543, 0.2549, 0.3251],
        [0.0543, 0.2549, 0.3251],
        [0.0543, 0.2549, 0.3251],
        [0.0543, 0.2549, 0.3251],
        [0.0543, 0.2549, 0.3251]])
tensor([0.0543, 0.2549, 0.3251])


### **그라디언트 추적 비활성화**
기본적으로는 모든 텐서가 `requires_grad=True`로 계산 기록을 추적하지만 그럴 필요가 없을때 `torch.no_grad()`나 `detach()`로 추적을 중지 할 수 잇다.

In [7]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

True
False
False


### **그래디언트 추적 비활성화 이유**
* __고정 된 매개변수__로 신경망의 일부 매개 변수를 표시합니다.
* 계산 속도를 올리기 위해서

### **계산 그래프에 대한 추가 정보**
개념적으로 오토그래드는 객체로 도니 DAG에 데이터 실행의 모든 작업을 유지한다.   
이 DAG에서 잎은 입력텐서, 뿌리는 출력텐서이다.   
**정방향 패스**에서 qutograd는 `요청된 작업 실행결과 텐서를 계산`, `DAG에서 작업의 기울기 기능을 유지합니다.`   
**역방향 패스** 는 DAG 루트에서 호출될 때 시작되며 기울기 계산(`.grad_fn`), 각 텐서의 `.grad` 속성에 누적, 체인 규칙으로 리프 텐서까지 전파등을 수행한다.  

**DAG***는 파이토치에서 동적이다. 또한 `.backword()` 호출 후 autograd는 새 그래프를 채우기 시작한다.   

In [10]:
inp = torch.eye(5, requires_grad=True)
out = (inp+1).pow(2)
out.backward(torch.ones_like(inp), retain_graph=True)
print("First call\n", inp.grad)
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nSecond call\n", inp.grad)
inp.grad.zero_()
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nCall after zeroing gradients\n", inp.grad)

First call
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])

Second call
 tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])

Call after zeroing gradients
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])


`backward`를 동일한 인수로 사용해도 그래디언트 값이 다르다.   
이것은 `backward` 전파이 파이토치가 __그래디언트 축적__하기 때문이다.__   
즉, 계산된 그래디언트 값이 게산 `grad` 그래프의 모든 리프 노드에 추가된다...   
적절한 그래디언트 계산을 위해서는 grad 이전에 속성을 0으로 해야한다